In [1]:
import pandas as pd

In [2]:
data_filename = "basketball.csv"
dataset = pd.read_csv(data_filename)

In [3]:
dataset.head(5)

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,Tue Oct 27 2015,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN
1,Tue Oct 27 2015,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN
2,Tue Oct 27 2015,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN
3,Wed Oct 28 2015,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN
4,Wed Oct 28 2015,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN


In [4]:
dataset = pd.read_csv(data_filename, parse_dates = ["Date"])
dataset.columns = ["Date", "Start(ET)", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "OT?", "Score Type", "Attned", "Notes"]

In [5]:
dataset.head(5)

,Date,Start(ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attned,Notes
0,2015-10-27,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN
1,2015-10-27,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN
2,2015-10-27,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN
3,2015-10-28,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN
4,2015-10-28,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN


In [6]:
print(dataset.dtypes)

Date            datetime64[ns]
Start(ET)               object
Visitor Team            object
VisitorPts               int64
Home Team               object
HomePts                  int64
OT?                     object
Score Type              object
Attned                   int64
Notes                   object
dtype: object


In [7]:
dataset["HomeWin"] = dataset["VisitorPts"] < dataset["HomePts"]
y_true = dataset["HomeWin"].values
dataset["HomeWin"].mean()

0.5942249240121581

In [8]:
from collections import defaultdict

In [9]:
won_last = defaultdict(int)
dataset["HomeLastWin"] = 0
dataset["VisitorLastWin"] = 0

In [10]:
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    dataset.at[index, "HomeLastWin"] = won_last[home_team]   
    dataset.at[index, "VisitorLastWin"] = won_last[visitor_team]  
    won_last[home_team] = int(row["HomeWin"])
    won_last[visitor_team] = 1-int(row["HomeWin"])

In [11]:
dataset.head(6)

,Date,Start(ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attned,Notes,HomeWin,HomeLastWin,VisitorLastWin
0,2015-10-27,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN,False,0,0
1,2015-10-27,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN,True,0,0
2,2015-10-27,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN,True,0,0
3,2015-10-28,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN,True,0,0
4,2015-10-28,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN,False,0,1
5,2015-10-28,7:30p,Utah Jazz,87,Detroit Pistons,92,Box Score,NaN,18434,NaN,True,1,0


In [12]:
dataset.iloc[1000:1005]

,Date,Start(ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attned,Notes,HomeWin,HomeLastWin,VisitorLastWin
1000,2016-03-15,7:00p,Denver Nuggets,110,Orlando Magic,116,Box Score,NaN,16988,NaN,True,0,0
1001,2016-03-15,8:30p,Los Angeles Clippers,87,San Antonio Spurs,108,Box Score,NaN,18418,NaN,True,1,0
1002,2016-03-16,7:00p,Oklahoma City Thunder,130,Boston Celtics,109,Box Score,NaN,18624,NaN,False,0,1
1003,2016-03-16,7:00p,Orlando Magic,99,Charlotte Hornets,107,Box Score,NaN,16148,NaN,True,0,1
1004,2016-03-16,7:00p,Dallas Mavericks,98,Cleveland Cavaliers,99,Box Score,NaN,20562,NaN,True,0,1


In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
clf = DecisionTreeClassifier(random_state = 14)

In [15]:
X_previouswins  = dataset[["HomeLastWin", "VisitorLastWin"]].values

In [16]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [17]:
scores = cross_val_score(clf, X_previouswins, y_true, scoring = 'accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores)*100))

Accuracy: 59.4%


/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [18]:
standings = pd.read_csv("standings.csv", skiprows=1)

In [19]:
standings.head()

,Rk,Team,Overall,Home,Road,E,W,A,C,SE,...,Post,≤3,≥10,Oct,Nov,Dec,Jan,Feb,Mar,Apr
0,1,Golden State Warriors,67-15,39-2,28-13,25-5,42-10,9-1,7-3,9-1,...,25-6,5-3,45-9,1-0,13-2,11-3,12-3,8-3,16-2,6-2
1,2,Atlanta Hawks,60-22,35-6,25-16,38-14,22-8,12-6,14-4,12-4,...,17-11,6-4,30-10,0-1,9-5,14-2,17-0,7-4,9-7,4-3
2,3,Houston Rockets,56-26,30-11,26-15,23-7,33-19,9-1,8-2,6-4,...,20-9,8-4,31-14,2-0,11-4,9-5,11-6,7-3,10-6,6-2
3,4,Los Angeles Clippers,56-26,30-11,26-15,19-11,37-15,7-3,6-4,6-4,...,21-7,3-5,33-9,2-0,9-5,11-6,11-4,5-6,11-5,7-0
4,5,Memphis Grizzlies,55-27,31-10,24-17,20-10,35-17,8-2,5-5,7-3,...,16-13,9-3,26-13,2-0,13-2,8-6,12-4,7-4,9-8,4-3


In [20]:
dataset["HomeTeamRanksHigher"] = 0
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    home_rank = standings[standings["Team"] == home_team]["Rk"].values[0]
    visitor_rank = standings[standings["Team"] == visitor_team]["Rk"].values[0]
    row["HomeTeamRanksHigher"] = int(home_rank < visitor_rank)
    dataset.at[index,"HomeTeamRanksHigher"] = int(home_rank < visitor_rank )

In [21]:
dataset.head()

,Date,Start(ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attned,Notes,HomeWin,HomeLastWin,VisitorLastWin,HomeTeamRanksHigher
0,2015-10-27,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN,False,0,0,1
1,2015-10-27,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN,True,0,0,0
2,2015-10-27,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN,True,0,0,1
3,2015-10-28,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN,True,0,0,1
4,2015-10-28,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN,False,0,1,0


In [22]:
X_homehigher =dataset[["HomeLastWin","VisitorLastWin","HomeTeamRanksHigher"]].values

In [23]:
clf = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(clf, X_homehigher, y_true, scoring = 'accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores)*100))

Accuracy: 60.9%


/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [24]:
last_match_winner = defaultdict(int)
dataset["HomeTeamWonLast"] = 0

In [25]:
for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team, visitor_team])) #Sort for a consistent ordering
    # Set in the row, who won thelast encounter
    home_team_won_last = 1 if last_match_winner[teams] == row["Home Team"] else 0 
    dataset.at[index, "HomeTeamWonLast"] = home_team_won_last
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner

In [26]:
X_lastwinner =dataset[["HomeTeamWonLast","HomeTeamRanksHigher","HomeLastWin","VisitorLastWin"]].values
clf = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(clf, X_lastwinner, y_true, scoring = 'accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores)*100))

Accuracy: 62.2%


/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [27]:
from sklearn.preprocessing import LabelEncoder

In [28]:
encoding = LabelEncoder()
encoding.fit(dataset["Home Team"].values)
home_teams = encoding.transform(dataset["Home Team"].values)
visitor_teams = encoding.transform(dataset["Visitor Team"].values)
X_teams = np.vstack([home_teams, visitor_teams]).T

In [29]:
from sklearn.preprocessing import OneHotEncoder

In [30]:
onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).todense()

/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [31]:
clf = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(clf, X_teams, y_true, scoring = 'accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores)*100))

Accuracy: 62.8%


/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
clf =RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring = 'accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores)*100))

/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWar

Accuracy: 65.3%


In [34]:
X_all = np.hstack([X_lastwinner, X_teams])
clf =RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_all, y_true, scoring = 'accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores)*100))

/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 63.3%


/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [35]:
from sklearn.model_selection import GridSearchCV

In [36]:
parameter_space = {"max_features": [2,10,'auto'], "n_estimators":[100, 200],"criterion":["gini", "entropy"], "min_samples_leaf":[2,4,6]}

In [39]:
clf =RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_*100))

/Users/Claire/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Accuracy: 67.4%


In [41]:
print(grid.best_estimator_)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=14, verbose=0, warm_start=False)
